In [1]:
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui
%gui tk
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))


In [2]:
# same problem as in 'Mixed'
source = sin(3.14*x)
ud = CoefficientFunction(5)
g = CoefficientFunction([y*(1-y) if bc=="left" else 0 for bc in mesh.GetBoundaries()])
lam = 10

In [3]:
order = 3
V = HDiv(mesh, order=order, flags = { "discontinuous" : True })
Q = L2(mesh, order=order-1)
F = FacetFESpace(mesh, order=order, dirichlet="bottom")
X = FESpace([V,Q,F])


In [4]:
sigma,u,uhat = X.TrialFunction()
tau,v,vhat = X.TestFunction()

a = BilinearForm(X)
a += SymbolicBFI(1/lam * sigma*tau + div(sigma)*v + div(tau)*u)
n = specialcf.normal(mesh.dim)
a += SymbolicBFI(sigma*n*vhat+tau*n*uhat, element_boundary=True)
a.Assemble()

f = LinearForm(X)
f += SymbolicLFI(-source*v)
f += SymbolicLFI(g*vhat, BND)
f.Assemble()

In [5]:
gfu = GridFunction(X)
inv = a.mat.Inverse(freedofs=X.FreeDofs())
gfu.vec.data = inv * f.vec


In [6]:
Draw (gfu.components[0], mesh, "sigma")
Draw (gfu.components[1], mesh, "u")